In [77]:
# Setup imports and logging
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [72]:
# Import data
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("data/boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("data/boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("data/boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [61]:
# Define feature columns
feature_cols = [tf.contrib.layers.real_valued_column(k)
                for k in FEATURES]

In [62]:
# Define regressor
regressor = tf.contrib.learn.DNNRegressor(
    feature_columns=feature_cols,
    hidden_units=[10, 10],
    model_dir='/tmp/boston_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11354ca90>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [73]:
# Define input_fn
def input_fn(data_set):
    feature_cols = {
        k: tf.constant(
            data_set[k].values,
            # NOTE: define shape explicitly to get rid of the warning
            # Input to shape should be a list
            shape=[data_set[k].size, 1]
        ) for k in FEATURES
    }
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

In [74]:
# Train the model
regressor.fit(input_fn=lambda: input_fn(training_set), steps=5000)

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x11354ca10>, 'hidden_units': [10, 10], 'feature_columns': (_RealValuedColumn(column_name='crim', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='zn', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='indus', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='nox', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='rm', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='dis', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='tax', dimension=1, default_value=None, dtype=tf.float32,

In [75]:
# Evaluate the model
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

loss_score = ev['loss']
print('Loss: {0:f}'.format(loss_score))

Loss: 11.534886


In [76]:
# Make Predictions
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
predictions = list(itertools.islice(y, 6))
print('Predictions: {}'.format(str(predictions)))

Predictions: [31.688829, 19.299904, 21.964277, 34.842587, 14.855656, 17.088512]
